In [1]:
using Pkg

Pkg.activate("../Project.toml")

using Random
import .Iterators: cycle, Cycle, take

using Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!
using JLD2
using CUDA

 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`


In [2]:
function Knet.KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Knet.Cptr, pointer(x))
    k = Knet.KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end

In [3]:
include("../coco2014.jl")
od = COCO14Data(
    "../dataset/images/overfit",
    "../val_labels.json",
    class_file="../class_mappings.json"
);

dtrn = minibatch(od.x[:, :, :, 1:6], od.y[1:6], 2; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(od.x[:, :, :, 10:11], od.y[10:11], 2; xsize = (416,416,3,:), xtype=Knet.atype());



# xx, yy = first(dtrn);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:15


In [7]:
save_data(od, "../ov_data.jld2")

┌ Info: Saved to: ../ov_data.jld2.
└ @ Main /home/bcs/Desktop/MSc/repos/yolo-julia/coco2014.jl:37


In [8]:
od_loaded = load_data("../ov_data.jld2")

┌ Info: Loaded: ../ov_data.jld2.
└ @ Main /home/bcs/Desktop/MSc/repos/yolo-julia/coco2014.jl:43


COCO14Data(Float32[0.6745098 0.6313726 … 0.99607843 0.99607843; 0.6784314 0.6784314 … 0.99607843 0.99607843; … ; 0.14509805 0.101960786 … 0.94509804 0.9529412; 0.1254902 0.09803922 … 0.972549 0.93333334]

Float32[0.53333336 0.48235294 … 1.0 1.0; 0.5372549 0.53333336 … 1.0 1.0; … ; 0.1764706 0.13725491 … 0.59607846 0.6039216; 0.16078432 0.13725491 … 0.61960787 0.5803922]

Float32[0.4862745 0.4392157 … 0.9764706 0.9764706; 0.49019608 0.4862745 … 0.9764706 0.9764706; … ; 0.18039216 0.13725491 … 0.57254905 0.57254905; 0.15686275 0.13333334 … 0.5882353 0.54901963]

Float32[0.654902 0.7607843 … 0.5058824 0.65882355; 0.8666667 0.972549 … 0.64705884 0.73333335; … ; 0.61960787 0.7137255 … 0.5411765 0.54509807; 0.654902 0.70980394 … 0.5882353 0.5686275]

Float32[0.6509804 0.72156864 … 0.36862746 0.6; 0.8784314 0.9607843 … 0.7058824 0.8352941; … ; 0.5882353 0.6901961 … 0.49803922 0.5176471; 0.5921569 0.64705884 … 0.54901963 0.54901963]

Float32[0.7490196 0.8117647 … 0.28235295 0.4509804; 0.909803

In [10]:
dtrn = minibatch(od_loaded.x[:, :, :, 1:6], od_loaded.y[1:6], 2; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(od_loaded.x[:, :, :, 10:11], od_loaded.y[10:11], 2; xsize = (416,416,3,:), xtype=Knet.atype());


xx, yy = first(dtrn)

(Float32[0.67058825 0.6862745 … 0.27058825 0.27058825; 0.6784314 0.69411767 … 0.27450982 0.2784314; … ; 0.7294118 0.7294118 … 0.77254903 0.7372549; 0.7294118 0.7294118 … 0.5058824 0.44313726]

Float32[0.5411765 0.5647059 … 0.29803923 0.28235295; 0.5568628 0.57254905 … 0.30588236 0.29803923; … ; 0.7176471 0.7176471 … 0.5176471 0.3882353; 0.7176471 0.7176471 … 0.23529412 0.18039216]

Float32[0.2901961 0.30980393 … 0.16078432 0.14901961; 0.30588236 0.31764707 … 0.16078432 0.16078432; … ; 0.59607846 0.59607846 … 0.52156866 0.42745098; 0.5882353 0.59607846 … 0.24705882 0.19607843]

Float32[0.6745098 0.6313726 … 0.99607843 0.99607843; 0.6784314 0.6784314 … 0.99607843 0.99607843; … ; 0.14509805 0.101960786 … 0.94509804 0.9529412; 0.1254902 0.09803922 … 0.972549 0.93333334]

Float32[0.53333336 0.48235294 … 1.0 1.0; 0.5372549 0.53333336 … 1.0 1.0; … ; 0.1764706 0.13725491 … 0.59607846 0.6039216; 0.16078432 0.13725491 … 0.61960787 0.5803922]

Float32[0.4862745 0.4392157 … 0.9764706 0.9764706; 0.

In [ ]:
Pkg.add("ProgressMeter")

In [ ]:
include("../models.jl")

Random.seed!(1)
darknet = Darknet("../yolov3.cfg"; verbose=false);


@diff a = darknet(xx, yy)

# a = darknet(xx, yy)

In [ ]:
bce(scores, labels; average=true)

In [ ]:
bce(out[5, :, :, :, :][:], convert(Array{Integer}, tobj)[:])

In [ ]:
tobj

In [ ]:
min.(max.(exp.(ps[3:4, :]), 0), 1000)  # custom clamp

In [ ]:
an = convert(Array{Bool}, a)

In [ ]:
vcat(an'...)

In [ ]:
function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end



In [ ]:

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 1; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 1; xsize = (416,416,3,:), xtype=Knet.atype());
include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

model = darknet

iters, trnloss, trnacc, tstloss, tstacc = train!(
    model, dtrn, dtst; 
    period=1, iters=520, lr=1e-2, optimizer=adam!);

In [ ]:
@load "first.jld2" darknet

In [ ]:
@diff a = darknet(x[:, :, :, 1:2], y[1:2])

In [ ]:
plot(iters, trnloss, label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, tstloss, label="test"))

plot(iters, round.(1 .- trnacc, digits=2), label="train", xlabel="Iterations", ylabel="Misclassification error")
display(plot!(iters, round.(1 .- tstacc, digits=2), label="test"))

In [ ]:

include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

l = darknet(xx, yy)


In [ ]:
sum(1.0 .- giou)

In [ ]:
inter = clamp.(min.(b1_x2, b2_x2) - max.(b1_x1, b2_x1), 0, 1e10) .* 
clamp.(min.(b1_y2, b2_y2) - max.(b1_y1, b2_y1), 0, 1e10);


In [ ]:
pbox[1, :] .- pbox[3, :] ./ 2

In [ ]:
clamp.(exp.(scores), 0, 1e3)

In [ ]:

b = Integer.(t[:, 1])
c = Integer.(t[:, 2])
gxy = t[:, 3:4]
gwh = t[:, 5:6]

In [ ]:
size(yolo_out[1])

In [ ]:
j

In [ ]:
j'

In [ ]:
tt[jj, :]

In [ ]:
tt = repeat(t, 3)

In [ ]:
tt[j]

In [ ]:
at'[jj]

In [ ]:
targets_reshaped .* gain

In [ ]:
wh1r = reshape(wh1, (:, 1, 2))

In [ ]:
wh12 = reshape(prod(wh1r, dims=3), (:, 1))

In [ ]:
wh2

In [ ]:
wh2r = reshape(wh2, (1, :, 2))

In [ ]:
wh22 = reshape(prod(wh2r, dims=3), (1, :))

In [ ]:
wh12 .+ wh22

In [ ]:
reshape(prod(a, dims=3), (3, 13))

In [ ]:
inter = build_targets(yolo_out, yy, darknet)


In [ ]:
targets = nothing 

for (i, t) in enumerate(yy[2:end])
    println(i)

end
gg = hcat(yy[2]...)
# gg = vcat(ones(1, 2), gg)

In [ ]:
length(yy[2])


In [ ]:
include("utils/utils.jl")

